# The John Conway's Game of Life simulation

<br>
 <div style="text-align: justify ">Game of Life (GoF) is a cellular automaton devised by the British mathematician John Horton Conway in 1970. The game is a zero-player game, meaning that its evolution is determined by its initial state, requiring no further input. One interacts with the Game of Life by creating an initial configuration and observing how it evolves, or, for advanced players, by creating patterns with particular properties.

The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, live or dead. Every cell interacts with its eight neighbours, which are the cells that are directly horizontally, vertically, or diagonally adjacent. At each step in time, the following transitions occur:</div>

<!-- <p style="text-align: center;"> [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) </p>
 -->
<!-- <h1><center> [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) </center></h1>
 -->


![SegmentLocal](Image/gol_intro.gif "segment") 
<center><a href="https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life" >Conway's Game of Life</a></center>


## Rules:

#### 1-Any live cell with fewer than two live neighbours dies, as if by needs caused by underpopulation.
#### 2-Any live cell with more than three live neighbours dies, as if by overcrowding.
#### 3-Any live cell with two or three live neighbours lives, unchanged, to the next generation.
#### 4- Any dead cell with exactly three live neighbours becomes a live cell.
The initial pattern constitutes the 'seed' of the system. The first generation is created by applying the above rules simultaneously to every cell in the seed – births and deaths happen simultaneously, and the discrete moment at which this happens is sometimes called a tick. (In other words, each generation is a pure function of the one before.) The rules continue to be applied repeatedly to create further generations until the world eventually converge. 

In [1]:
# We use Pygame to visualize the simulation

import pygame
import numpy as np 
import matplotlib as mpl
import matplotlib.backends.backend_agg as agg
import multiprocessing 
import os

from time import sleep 
from time import time as ttime
from matplotlib import pyplot as plt
from organism import *
from support import search


## To Make the Screen Background Black
plt.style.use('dark_background')
mpl.use("Agg")

pygame 2.2.0 (SDL 2.0.22, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


## Game Configuration

 Here we set the basic world (screen) configurations :
- We use the pygame library in order to make several screens, one for the main game ("Sub2" in right_screen) , the other one for having the live Density over Iteration plot (Sub1 in top_left) and the last one to report us some statistics about the world (Sub0 in buttom_left).
<br/>
-  We also creat a coloring system for the cells in a way that there are 3 phase for each cell:
  - <font color='green'>Child (between age 0 to 50 )</font>
<br/>
  - <font color='red'>Young (between age 50 to 100 )</font>
 <br/>
  - <font color='gray'>Gray (over the age of 100 )</font>

In [2]:
## Start the Game
pygame.init()

width,height = pygame.display.get_desktop_sizes()[0]
pixel = 30                                                     # Specifies the size of the world (Cells by Cells)
px = 1/plt.rcParams['figure.dpi']

# World screen
width_sub2 = height            
height_sub2 = height
world = np.zeros((height_sub2//pixel, width_sub2//pixel, 2)).astype(int)

# Plot window
width_sub1 = width - height    
height_sub1 = height // 3 + 50
fig,ax = plt.subplots(figsize=(width_sub1*px, height_sub1*px))

# Raw data to display
width_sub0 = width_sub1        
height_sub0 = height - height_sub1

# Population display
fign, axesn = plt.subplots(4, len(Organisms)//4, figsize=(width_sub0*px, height_sub0*px))

# get a copy of an given array collapsed into one dimension
axes = axesn.flatten()                        

# Basic color
white = (255, 255, 255)
black = (0, 0, 0)
green = (0, 255, 0)
red = (255, 0, 0)
gray = (80, 80, 80)
light_blue = (93, 188, 194)

# Customized colors
n = 50  # palette lenght

palette1 = [(int(i*255/n),255,int(i*255/n)) for i in range(n,-1,-1)]
palette2 = [(int((n-i)*255/n),int(i*255/n),0) for i in range(n,-1,-1)]
palette3 = [(255 - int((n-i)*175/n),int((n-i)*80/n),int((n-i)*80/n)) for i in range(n,-1,-1)]

palette = palette1 + palette2 + palette3

# Pygame configuration
font = pygame.font.SysFont('dejavusans', 24)
screen = pygame.display.set_mode((width, height),pygame.RESIZABLE) 
title = pygame.display.set_caption("Game of Life")

# Camera configuration
canvas = pygame.Surface((width, height))

p1_camera = pygame.Rect(0,0,width_sub1,height_sub1)
p2_camera = pygame.Rect(width_sub1,0,width_sub2,height_sub2)
p0_camera = pygame.Rect(0,height_sub1,width_sub0,height_sub0)

sub1 = canvas.subsurface(p1_camera)
sub2 = canvas.subsurface(p2_camera)
sub0 = canvas.subsurface(p0_camera)
sub4 = canvas.subsurface(p0_camera)


## Stat configuration
- Here we have the initial configuration for showing some stats in the "Sub0" Section 

In [3]:
density = []           # The overall alive cells per world size

born = 0               # Counts the total/current born

death = 0              # Counts the total/curren death

FPS = 0                # To show how fast the game is

converged = False      # It's true when the simulation reach the convergence

periodic = True        # If it's false the world has the borders

run = True             # It's true when the simulation is running

time_to_sleep = 0.3    # It regulates the default speed of the simulation

pause = True           # When it's true we can edit and reset the world

statmenu = False       # To display the information about the population composition

## Main functions of The Game

- The **"next_gen"** function is the very heart of this simulation: it is the implementation of the rules, and it modifies the age of the living cells

In [4]:
def next_gen(world, periodic = True):
    global born
    global death
    
    height, width = world.shape[:-1] 
    new_world = np.zeros((height, width, 2)).astype(int) # make a new world that we want to edit
    
    if periodic:
        for i in range(height): 
            for j in range(width):  

                cell = world[i][j][0]
                neighbours_count = np.array([
                        world[i_r][j_r][0]
                        for i_r in ((i-1)%height, i, (i+1)%height) #periodic condition applied
                        for j_r in ((j-1)%width, j, (j+1)%width)
                ]).sum() - cell

                if cell:
                    if neighbours_count in (2, 3):                 #implementation of the rules
                        new_world[i][j][0] = 1
                        new_world[i][j][1] = world[i][j][1] + 1
                    else:
                        new_world[i][j][1] = 0
                        death += 1

                elif (not cell) and neighbours_count == 3:
                    new_world[i][j][0] = 1
                    new_world[i][j][1] = 0
                    born += 1
    
    else:
        pad_world = np.pad(world[:,:,0], 1)                        #pad to represent a closed world
        
        for i in range(1, height+1): 
            for j in range(1, width+1):  

                cell = pad_world[i][j]
                neighbours_count = pad_world [i-1:i+2, j-1:j+2].sum() - cell

                if cell:
                    if neighbours_count in (2, 3):
                        new_world[i-1][j-1][0] = 1
                        new_world[i-1][j-1][1] = world[i-1][j-1][1] + 1
                    else:
                        new_world[i-1][j-1][1] = 0
                        death += 1

                elif (not cell) and neighbours_count == 3:
                    new_world[i-1][j-1][0] = 1
                    new_world[i-1][j-1][1] = 0
                    born += 1
            
    return new_world

## Helpful Function : part 1
 Here we define few functions to help us showing what we need in different part of the screen
<br/>

 - **display_data :** Function to modify the stats in each iteration
 <br/>

 - **make_plot :** Display the Density plot in "Sub1" Canvas
 <br/>

 - **show_stat :** Display the simulation data stats in "Sub0" Canvas

In [5]:
## This funtion Modifies the stats in each iteration
def display_data():
    
    sub0.fill(black)
    
    time = len(density)
    
    child = cell[age < n].sum()
    young = cell[age < 2*n].sum() - child
    old = cell[age >= 2*n].sum()
    
    current_alive = child + young + old
    current_dead = cell.size - current_alive
    
    if len(age[cell == 1]) > 0:                                # to avoid the error "mean of an empty list"
        average_age = np.mean(age[cell == 1]).round(2)        
    else:
        average_age = "Empty World" 

## The main Dictionary ro display in "Sub0"
    data = {
        "Boundry": ("Periodic" if periodic else "Closed", white),
        "FPS": (FPS ,white),
        "Generation": (time, white),
        "Child Population": (child, green),
        "Young Population": (young, red),
        "Old Population": (old, gray),
        "Currently Dead": (current_dead, white),
        "Total Birth Count": (born, white),
        "Total Death Count": (death, white),
        "Average Age": (average_age, white),
        "Converged": ('Y (Press S)', light_blue) if converged else ("N", light_blue)}

## Now we want to display the informations in Data
    h0 = (height_sub0-100) / (len(data))
    h = 20
    
    for key, item in data.items():
        stat = font.render(key, True, item[1])
        value = font.render(str(item[0]), True, item[1])
        sub0.blit(stat, (50, h))
        sub0.blit(value, (width_sub0 - 200, h))
        h += h0

## This Function Plots the Density Plot over each Iteration  in "sub1"      
def make_plot(fig,ax,count):
    ax.clear()
    ax.plot(np.arange(0, len(density)), count, linewidth=3, color='m')
    ax.fill_between(np.arange(0, len(density)), np.zeros(len(count)), count, color='m', alpha=0.5)
    ax.set_ylabel("Density of Alive Cells", fontsize=15)
    ax.grid()    
    canvas1 = agg.FigureCanvasAgg(fig)
    canvas1.draw()
    renderer = canvas1.get_renderer()
    raw_data = renderer.tostring_rgb()
    size = canvas1.get_width_height()
    sub1 = pygame.image.frombuffer(raw_data, size, "RGB")
    return sub1


## This function plots the organism and the number of times the organisms has been detected in the world (when presses s buttom in "sub0")
def show_stat():                          
    for i, k in enumerate(Organisms):
        last = world[:, :, 0]
        
## Display population
        org_count = sum([search(last, org) for org in Organisms[k]])
        axes[i].set(adjustable='box', aspect='equal')                              # set a boxshape and equal aspects in order to have better looks
        axes[i].pcolormesh(Organisms[k][0][::-1,:], edgecolors='k', linewidth=2)   # create a pseudocolor plot with a non-regular rectangular grid
        axes[i].tick_params(left = False, right = False , labelleft = False ,      # remove ticks
                    labelbottom = False, bottom = False)
        axes[i].set_title(f"{k}: {org_count}", fontsize=10)                        # set titles
        
    canvas_s = agg.FigureCanvasAgg(fign)
    canvas_s.draw()
    renderer = canvas_s.get_renderer()
    raw_data_s = renderer.tostring_rgb()
    size = canvas_s.get_width_height()
    fig.tight_layout()
    sub4 = pygame.image.fromstring(raw_data_s, size, "RGB")
    
    # To enclose the population information in a box and to print the output:
    pygame.draw.rect(sub4, white, (0, 0, width_sub1, 2))
    pygame.draw.rect(sub4, white, (width_sub0-2, 0, 2, height_sub0))
    screen.blit(sub4, (0, height_sub1))


## Helpful Function : part 2
#### In the following block we have few more functions :
<br/>

 - **user_edit :** Permit the user to edit the world, creat new alive cells or kill other cells 
 <br/>

 - **update_screen :** To be implemented in the main loop in order to update the whole screen (Sun2)
 <br/>

 - **default :** Refreshes some of the Stats

In [6]:
## Make alive or kill cell using the mouse function
def user_edit(world):
    # if left click: make cell alive
    if pygame.mouse.get_pressed()[0]:         # if left click is pressed
        x, y = pygame.mouse.get_pos()         # take the cursor position and try to highlight on grid
        if x>= width_sub1:
            x -= width_sub1
            world[y//pixel][x//pixel][0] = 1
            world[y//pixel][x//pixel][1] = 0
        
    # if right click: make cell dead
    if pygame.mouse.get_pressed()[2]:
        x, y = pygame.mouse.get_pos()
        if x>= width_sub1:
            x -= width_sub1
            world[y//pixel][x//pixel][0] = 0
            world[y//pixel][x//pixel][1] = 0
    
    return world

## Just Updating the whole world and screen
def update_screen(world):
    sub2.fill(black)                                      # Update the screen
    height, width = world.shape[:-1]                      
    for i in range(height):                               # Updating the cells
        for j in range(width):
            color_index = world[i][j][1]
            if world[i][j][0]:
                pygame.draw.rect(sub2, palette[min(color_index, len(palette)-1)], (j*pixel, i*pixel, pixel, pixel))  # Update the world screen (Sub2) in new iteration
                
# An utility function to reset the world
def default(density,ax,born,death,FPS):
    ax.clear()
    return [],0,0,0


## The Main Simulation Loop
 In the following block we start the main pygame loop, in which all the functions defined above were called.
 There are several part here all working together which can be devided into these four sections :

 - #### Updating Stat Part : 
 some of the stas are getting updated in each iteration (for example the number of alive cells)
 <br/>
 - #### FPS part : 
 It is being controlled by scrolling the mouse (mouse wheel) and simply just shows a measure for speed of the game. we put it in between 0 and .  

 - #### The Keyboard buttoms : 
 There are several keyboard buttoms which each do something in the game. here are the list for them :
 <br/>

  - **Pause (press "middle mouse" ) :** Puases the game and you can see the stats, clear the density chart and edit the world
  - **Random (press "R") :** Creats a random world each time the player presses it.
  - **Border (press "P") :** simply opens (periodic border) and closes the border (closed border).
  - **Clear (press "C") :** Clears the main screen (sub2).
  - **Show Stats (press "S") :** When in Puase, shows the different organisms counts.
  - **Clear Density (press "D" ):** Clear the density plot. it works after unpuasing the game.
  - **Scape ("esc") :** Quit the game.
 - #### The Updating the Screen :  
 Using these two functions "update_screen()" and "pygame.display.update()" to recreat the whole features and screens of the game

 
 

In [7]:
try:
    while run:
        a = ttime()                                               # start a clock to compute FPS
        update_screen(world) 
        if not pause:
            cell = world[:, :, 0]                                 # First layer of our matrix that show live/dead cells
            age = world[:, :, 1]                                  # Secondo layer to display the cells age
            
            density.append(cell.sum()/(cell.size))                # a matrix for the plot
            sub1 = make_plot(fig,ax,density)                      # Draw a plot on sub1
            
            display_data()                                        # Display the simulation info
            
            world = next_gen(world, periodic)                     # Compute the next step
            sleep(time_to_sleep)                                  # An utility function to control the speed game
            
            
        if pause and not statmenu:
            world = user_edit(world)    
            
        # These are the boxes definition and the drawing
        pygame.draw.rect(sub0, white, (0, 0, width_sub1, 2))
        pygame.draw.rect(sub0, white, (width_sub0-2, 0, 2, height_sub0))
        pygame.draw.rect(sub1, white, (width_sub1-2, 0, 2, height_sub1))
        
        screen.blit(sub1, (0,0))
        screen.blit(sub2, (width_sub1, 0))
        screen.blit(sub0, (0, height_sub1))
        
        if statmenu and pause:
            show_stat()
            
        pygame.display.update()
        
        # Finally the convergence condition
        if len(density) > 90 and (all(c==density[-1] for c in density[-10:]) or all(c==density[-30] for c in density[-90::30])) and not pause:
            converged = True
        else:
            converged = False
            
        # Now the user input handling speed (FPS)
        for event in pygame.event.get():
            if event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 2:                                 # Mouse wheel click
                    pause = not pause
                if event.button == 4:                                 # Mouse wheel scroll up
                    time_to_sleep = max(0, time_to_sleep + 0.05)
                    time_to_sleep = min(time_to_sleep, 1)
                if event.button == 5:                                 # Mouse wheel scroll down
                    time_to_sleep = max(0, time_to_sleep - 0.05)
                    time_to_sleep = min(time_to_sleep, 1)
            if event.type == pygame.QUIT:
                pygame.display.quit()
                pygame.QUIT
                run = False
                
            if event.type == pygame.KEYDOWN:
                if event.key == 27: # esc
                    pygame.display.quit()
                    pygame.QUIT
                    run = False
                if event.key == 114: # R
                    world = np.zeros((height_sub2//pixel, width_sub2//pixel, 2)).astype(int)
                    M = np.random.randint(0, 2, (height_sub2//pixel, width_sub2//pixel)).astype(int)
                    world[:, :, 0] = M.copy()
                if event.key == 99: # C -> This is a soft reset
                    world = np.zeros((height_sub2//pixel, width_sub2//pixel, 2)).astype(int)
                    born = 0
                    death = 0
                if event.key == 112: # P
                    periodic = not periodic
                if pause:
                    if event.key == 115: # S
                        statmenu = not statmenu
                    if event.key == 100: # D -> This is the hard reset
                        density,born,death,FPS = default(density,ax,born,death,FPS)
                        world = np.zeros((height_sub2//pixel, width_sub2//pixel, 2)).astype(int)
                        
                        
        b = ttime()
        
        FPS = np.round(1/(b-a),2)
    
        
except Exception as e:
    print(e)
    pygame.display.quit()
    pygame.QUIT